In [2]:
import asyncio
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.genai import types

# Import your local modules
from veganflow_ai.agents.orchestrator import create_store_manager
from veganflow_ai.tools.retail_database_setup import setup_retail_database

# --- 1. Setup Infrastructure ---
print("📦 Resetting Retail Database...")
setup_retail_database()

print("🧠 Initializing Services...")
# We use InMemory services for local testing to keep it fast
memory_service = InMemoryMemoryService()
session_service = InMemorySessionService()
user_id = "test_manager"
session_id = "orchestrator_test_session"

# --- 2. Seed Strategic Context ---
# The Orchestrator (via Procurement) needs to know price targets
print("🌱 Seeding Strategy...")
seeder = LlmAgent(name="seeder", model=Gemini(model="gemini-2.0-flash"), instruction="Act as scribe.")
seeder_runner = Runner(agent=seeder, session_service=session_service, app_name="seeder_app")

# Create session for seeding
await session_service.create_session(app_name="seeder_app", user_id=user_id, session_id="seed_session")

# Strategy: Oat Milk target is $3.40
strategy = "Strategic Goal: For 'Oat Barista Blend', target price is $3.40. Do not pay over $3.60."

async for _ in seeder_runner.run_async(
    user_id=user_id, session_id="seed_session", new_message=types.Content(parts=[types.Part(text=strategy)])
): pass

# Ingest into memory
seed_session = await session_service.get_session(app_name="seeder_app", user_id=user_id, session_id="seed_session")
await memory_service.add_session_to_memory(seed_session)

# --- 3. Initialize Orchestrator ---
print("🤖 Waking up Store Manager...")
orchestrator = create_store_manager()

# Create session for the main test
await session_service.create_session(
    app_name="store_manager_test", 
    user_id=user_id, 
    session_id=session_id
)

# Configure Runner with ALL services
runner = Runner(
    agent=orchestrator,
    session_service=session_service,
    memory_service=memory_service, # Critical for sub-agents to read strategy
    app_name="store_manager_test"
)

# --- 4. Run the Complex Scenario ---
print("\n" + "="*60)
print("🧪 SCENARIO: Analyze Risks & Resolve Them")
print("="*60)

# This single command should trigger:
# 1. Orchestrator delegates to Shelf Monitor -> Finds Oat Milk is critical
# 2. Orchestrator logic sees critical status -> Delegates to Procurement
# 3. Procurement checks Memory -> Finds $3.40 target
# 4. Procurement checks Vendors -> Finds Earthly/Clark/LCG
# 5. Procurement negotiates via A2A -> Buys stock
# 6. Orchestrator summarizes result
query = "Analyze inventory risks and resolve any critical stockouts immediately."

await runner.run_debug(query)

App name mismatch detected. The runner is configured with app name "seeder_app", but the root agent was loaded from "/Users/karthicksothivelr/Downloads/Autonomous_Supply_Chain_Intelligence/venv/lib/python3.13/site-packages/google/adk/agents", which implies app name "agents".


📦 Resetting Retail Database...
✅ Database 'veganflow_store.db' rebuilt with 21 products and 16 competing offers.
   - CRITICAL SCENARIO: Oat Barista Blend has 0.8 days supply.
🧠 Initializing Services...
🌱 Seeding Strategy...


App name mismatch detected. The runner is configured with app name "store_manager_test", but the root agent was loaded from "/Users/karthicksothivelr/Downloads/Autonomous_Supply_Chain_Intelligence/venv/lib/python3.13/site-packages/google/adk/agents", which implies app name "agents".


🤖 Waking up Store Manager...

🧪 SCENARIO: Analyze Risks & Resolve Them

 ### Created new session: debug_session_id

User > Analyze inventory risks and resolve any critical stockouts immediately.
shelf_monitor > OK. I see a critical stockout risk: 'Oat Barista Blend' is critically low with only 12 units, and will stock out in less than a day. I will transfer this to the Procurement Negotiator to resolve.



/Users/karthicksothivelr/Downloads/Autonomous_Supply_Chain_Intelligence/veganflow_ai/agents/procurement.py:70: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  remote_agent = RemoteA2aAgent(
App name mismatch detected. The runner is configured with app name "procurement_negotiation_task", but the root agent was loaded from "/Users/karthicksothivelr/Downloads/Autonomous_Supply_Chain_Intelligence/venv/lib/python3.13/site-packages/google/adk/agents", which implies app name "agents".



🔄 [A2A] Initiating Handshake with http://localhost:8003...
📨 [A2A] Sending Offer: 210x Oat Barista Blend @ $2.93...


/Users/karthicksothivelr/Downloads/Autonomous_Supply_Chain_Intelligence/venv/lib/python3.13/site-packages/google/adk/agents/remote_a2a_agent.py:389: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  converted_parts = self._genai_part_converter(part)
/Users/karthicksothivelr/Downloads/Autonomous_Supply_Chain_Intelligence/venv/lib/python3.13/site-packages/google/adk/a2a/converters/event_converter.py:239: UserWarning: [EXPERIMENTAL] convert_a2a_message_to_event: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethem

Tb [A2A] Vendor Replied: We accept your offer. We can supply 210 units of Oat Barista Blend at $2.93. Delivery in 2 days.

procurement_negotiator > Deal secured. The critical stockout of 'Oat Barista Blend' is resolved. I've secured 210 units from Clark Distributing at $2.93 each, and they'll be delivered in 2 days.
   🧠 [Auto-Memory] Ingesting session insights...


[Event(model_version='gemini-2.5-pro', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={
           'agent_name': 'shelf_monitor'
         },
         id='adk-2b0c1bed-f95b-4cae-84aa-9830812db49f',
         name='transfer_to_agent'
       ),
       thought_signature=b'\n\xdf\n\x01\xe3\xf1\xff^\xcc\xa4)\\\xcch\xb9\xba\xdc`I\xdc\xeeY;\x14~\xc1l4U\x91.{h\xcc;\x7fGVo\x07l\x81\x10\x88\xc7T\xb07\x0cI\xb9e\xfc\x1c\xd3\xe2\xb96\x02\x05"\xbe\x9es6\xc8\xe5\xa7\xba\xab\x13\x0fo\x02,\xbf\xe4\xab/C\x89\xba\xe9\x00\xbc\x83\x18\xab\xfa\xe9D{\xd6P\xdd\'\x10...'
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=11,
   candidates_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       tok